In [ ]:
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil

import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
from ROOT import larocv

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# make a fake image...
a=np.zeros((485,485))

ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

ctor.clear()
    
pt_x=[]
pt_y=[]

ctor.push_back(ROOT.cv.Point_("int")(25,225))
ctor.push_back(ROOT.cv.Point_("int")(50,225))
ctor.push_back(ROOT.cv.Point_("int")(50,229))
ctor.push_back(ROOT.cv.Point_("int")(29,229))

ctor.push_back(ctor[0])
for pt in ctor:
    pt_x.append(pt.x)
    pt_y.append(pt.y)
    plt.plot(pt_x,pt_y,'o-',color='black',lw=1)

ax=plt.gca()
#ax.set_xlim(0,450)
#ax.set_ylim(0,450)
plt.grid()
plt.show()

# make a path representing this contour
path_list1 = []
for pt in ctor:
    path_list1.append((pt.x,pt.y)) 


In [ ]:
path1 = matplotlib.path.Path(path_list1)

img1 = path1.contains_points(np.row_stack(np.where(a==0)).T).reshape(485,485).astype(np.uint8)

img1=np.where(img1>0,img1+80,0)
img2 = img1.copy()
img3 = img1.copy()

M = cv2.getRotationMatrix2D((485/2,485/2),20,1)

#rotate 1
img1 = cv2.warpAffine(img1,M,(485,485))

#get nonzero for 1, and move it...
nzero=np.row_stack(np.where(img1>0)).T
img1[nzero[:,0],nzero[:,1]]=0
img1[nzero[:,0]+20,nzero[:,1]+75]=80

M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
#rotate 2
img2 = cv2.warpAffine(img2,M,(485,485))

#flip 2
img2 = img2[:,::-1]

#get nonzero for 2, and move it...
nzero=np.row_stack(np.where(img2>0)).T
img2[nzero[:,0],nzero[:,1]]=0
img2[nzero[:,0]+20,nzero[:,1]-179]=80

#rotate 3
M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
img3 = cv2.warpAffine(img3,M,(485,485))

#get nonzero for 3, and move it...
nzero=np.row_stack(np.where(img3>0)).T
img3[nzero[:,0],nzero[:,1]]=0
img3[nzero[:,0]+100,nzero[:,1]+148]=80

#add them...
img = img1+img2+img3

plt.imshow(img,cmap='Greys',interpolation='none')
ax=plt.gca()
ax.vlines(227,0,485,color='orange',lw=2,alpha=0.5)
ax.text(50,50,"Fake Image",fontsize=20,fontweight='bold',color='red')
fig=plt.gcf()
fig.set_size_inches(20,12)

plt.rc('grid', linestyle="-", color='black')

mjx = mjy = 10.;
mix = miy = 1.

major_ticks_x = np.arange(0,485+mjx/2,mjx)                                                                                                                                                                                    
major_ticks_y = np.arange(0,485+mjy/2,mjy)                                                                                                                                                                                    
minor_ticks_x = np.arange(0+mix/2,485+mix*1.5,1)                                                                                                                                                                            
minor_ticks_y = np.arange(0+miy/2,485+miy*1.5,1)                                                                                                                                                                            

ax.set_xticks(major_ticks_x)                                                                                                                                                                                            
#ax.set_xticks(minor_ticks_x, minor=True)                                                                                                                                                                                
ax.set_yticks(major_ticks_y)                                                                                                                                                                                            
#ax.set_yticks(minor_ticks_y, minor=True)                                                                                                                                                                                
ax.grid(which='minor')                                                                                                                                                                                                  
ax.set_xticklabels(major_ticks_x.astype(np.uint16),rotation=45,fontsize=10,minor=False)                                                                                                                                                   
ax.set_yticklabels(major_ticks_y.astype(np.uint16),rotation=45,fontsize=10,minor=False) 

plt.grid()
plt.show()

In [ ]:
iom = larcv.IOManager(larcv.IOManager.kWRITE)
iom.set_verbosity(0)
iom.set_out_file("/tmp/aho.root")
iom.initialize()
fake_evimg=iom.get_data(larcv.kProductImage2D,"test")

img2d1 = larcv.as_image2d(img1)
img2d2 = larcv.as_image2d(img2)
img2d3 = larcv.as_image2d(img3)

imeta1 = larcv.ImageMeta(485,2910.0,
                   485,485,
                   166.0,5424.0,
                   0)

imeta2 = larcv.ImageMeta(485,2910.0,
                   485,485,
                   621.0,5424.0,
                   1)

imeta3 = larcv.ImageMeta(485,2910.0,
                   485,485,
                   343.,5424.0,
                   2)

img2d1 = larcv.Image2D(imeta1,img2d1.as_vector())
img2d2 = larcv.Image2D(imeta2,img2d2.as_vector())
img2d3 = larcv.Image2D(imeta3,img2d3.as_vector())


fake_evimg.Append(img2d1)
fake_evimg.Append(img2d2)
fake_evimg.Append(img2d3)

iom.set_id(0,0,1)
iom.save_entry()
iom.finalize()

In [ ]:
proc = larcv.ProcessDriver("ProcessDriver")
proc.configure("linear_track_cluster_unit.fcl")
f=ROOT.std.string("/tmp/aho.root")
flist=ROOT.std.vector('string')()
flist.push_back(f)
proc.override_input_file(flist)
proc.initialize()
larbysimg = proc.process_ptr(0)

In [ ]:
proc.batch_process(0,1)

In [ ]:
mgr=larbysimg.Manager()
img_v = []
pygeo = geo2d.PyDraw()

In [ ]:
for mat in mgr.InputImages():
    img_v.append(pygeo.image(mat))

In [ ]:
plane=0
for mat in mgr.InputImages():
    img_v.append(pygeo.image(mat))
    shape_img=np.where(img_v[plane]>0.0,1.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()
    plane+=1

In [ ]:
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10

dm=mgr.DataManager()
data=dm.Data(0)
lintrk_v=data.get_clusters()

for strack in lintrk_v:
    
    # the only good one...
    
    e13d=strack.edge1
    for plane in xrange(3):
        strack2d = strack.get_cluster(plane)
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        shape_img = img_v[plane]
        shape_img=np.where(img_v[plane]>0.0,1.0,0.0).astype(np.uint8)
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        if strack2d.ctor.size()>0:
            ctor = [[pt.x,pt.y] for pt in strack2d.ctor]
            ctor.append(ctor[0])
            ctor=np.array(ctor)

            plt.plot(ctor[:,0],ctor[:,1],'-o',lw=3)

            e1=strack2d.edge1
            e2=strack2d.edge2
            
            plt.plot(e1.x,e1.y,'*',color='orange',markersize=20)
            plt.plot(e2.x,e2.y,'*',color='yellow',markersize=20)
        
            vtx2d=e13d.vtx2d_v[plane]
            pt=vtx2d.pt
            plt.plot(pt.x,pt.y,'o',color='green',markersize=40,alpha=0.7)
        
        ax.set_aspect(1.0)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        plt.xlabel('Time [6 ticks]',fontsize=20)
        plt.ylabel('Wire [2 wires]',fontsize=20)
        plt.tick_params(labelsize=20)
        ax.set_aspect(0.8)
        plt.show()